In [ ]:
# cpu_log_sqlite_min_full.py
# 1秒おきに観測し、CSVには全項目、SQLiteには
# (1) 4列の cpu_metrics_min と (2) 全項目の cpu_metrics_full を保存。
# 欠損は DB では NULL、CSV では -1 表示。
# 既存DBは削除（ロック時は退避改名）。

import json, os, socket, sqlite3, time, sys, csv, signal, atexit, math
from datetime import datetime, timezone
import psutil

# ===== 設定 =====
DB_PATH       = "cpu_metrics_2.db"
INTERVAL      = 1.0
BATCH_COMMIT  = 10
# ===============

# 標準出力（Windows対策）
try:
    sys.stdout.reconfigure(encoding="utf-8", newline="\n")
except Exception:
    pass

# 既存DB削除（ロック中は退避改名）
if os.path.exists(DB_PATH):
    try:
        os.remove(DB_PATH)
        print(f"[INIT] removed: {DB_PATH}", file=sys.stderr)
    except PermissionError:
        base, ext = os.path.splitext(DB_PATH)
        bak = f"{base}-{int(time.time())}.bak{ext}"
        try:
            os.replace(DB_PATH, bak)
            print(f"[INIT] renamed locked DB -> {bak}", file=sys.stderr)
        except Exception as e:
            print(f"[WARN] backup failed: {e}", file=sys.stderr)
    except Exception as e:
        print(f"[WARN] remove failed: {e}", file=sys.stderr)

# 温度取得（任意）
try:
    import wmi  # type: ignore
except Exception:
    wmi = None

HOSTNAME = socket.gethostname()
USERNAME = os.environ.get("USERNAME") or os.environ.get("USER")
STOP = False

def _stop(*_):
    global STOP
    STOP = True

# 安全停止
signal.signal(signal.SIGINT, _stop)
for sig in ("SIGTERM", "SIGBREAK"):
    if hasattr(signal, sig):
        try: signal.signal(getattr(signal, sig), _stop)
        except Exception: pass

def now_iso() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%fZ")

def _f(x):
    if x is None: return None
    try:
        xx = float(x)
        return None if math.isnan(xx) or math.isinf(xx) else xx
    except Exception:
        return None

def _i(x):
    if x is None: return None
    try:
        return int(x)
    except Exception:
        try:
            xx = float(x)
            return None if math.isnan(xx) or math.isinf(xx) else int(xx)
        except Exception:
            return None

def _json(v):
    # None/空配列/空辞書/JSON化失敗は None（DBでNULL）
    if v is None: return None
    if isinstance(v, (list, dict)) and len(v) == 0: return None
    try:
        return json.dumps(v, ensure_ascii=False)
    except Exception:
        return None

# --- 温度取得 ---
def _wmi_values(ns: str, ok) -> list:
    try:
        c = wmi.WMI(namespace=ns)  # type: ignore
        return [s.Value for s in c.Sensor() if ok(s)]
    except Exception:
        return []

def get_cpu_temp_c():
    # 1) psutil
    try:
        temps = getattr(psutil, "sensors_temperatures", lambda: {})()
        for _, arr in temps.items():
            for t in arr:
                cur = getattr(t, "current", None)
                name = (getattr(t, "label", None) or getattr(t, "sensor", None) or "").lower()
                if cur is None: continue
                if ("cpu" in name or "package" in name) and -50 < cur < 150:
                    return _f(cur)
        for _, arr in temps.items():
            for t in arr:
                cur = getattr(t, "current", None)
                if cur is not None and -50 < cur < 150:
                    return _f(cur)
    except Exception:
        pass
    # 2) Windows: LHM/OHM/ACPI
    if wmi is not None:
        try:
            vals = _wmi_values("root\\LibreHardwareMonitor",
                               lambda s: getattr(s, "SensorType", "") == "Temperature" and "cpu" in (s.Name or "").lower())
            if not vals:
                vals = _wmi_values("root\\OpenHardwareMonitor",
                                   lambda s: getattr(s, "SensorType", "") == "Temperature" and "cpu" in (s.Name or "").lower())
            if vals:
                try:
                    v = max([float(v) for v in vals if v is not None])
                    return _f(v if -50 < v < 150 else None)
                except Exception:
                    pass
        except Exception:
            pass
        try:
            c = wmi.WMI(namespace="root\\wmi")  # type: ignore
            rows = c.MSAcpi_ThermalZoneTemperature()
            cs = []
            for r in rows:
                try: cs.append(r.CurrentTemperature/10 - 273.15)
                except Exception: pass
            if cs:
                v = max(cs)
                return _f(v if -50 < v < 150 else None)
        except Exception:
            pass
    return None

# --- DB スキーマ ---
DDL_MIN = """
CREATE TABLE IF NOT EXISTS cpu_metrics_min (
  ts               TEXT    NOT NULL,
  cpu_percent      REAL,
  cpu_ctx_switches INTEGER,
  temp_c           REAL
);
CREATE INDEX IF NOT EXISTS idx_cpu_metrics_min_ts ON cpu_metrics_min(ts);
"""

DDL_FULL = """
CREATE TABLE IF NOT EXISTS cpu_metrics_full (
  ts               TEXT    NOT NULL,
  cpu_percent      REAL,
  temp_c           REAL,

  cpu_user_pct     REAL,
  cpu_system_pct   REAL,
  cpu_idle_pct     REAL,
  cpu_iowait_pct   REAL,

  cpu_ctx_switches INTEGER,
  cpu_interrupts   INTEGER,
  cpu_soft_intr    INTEGER,
  cpu_syscalls     INTEGER,

  cpu_logical      INTEGER,
  cpu_physical     INTEGER,

  cpu_freq_mhz       REAL,
  cpu_freq_min_mhz   REAL,
  cpu_freq_max_mhz   REAL,

  per_core_cpu     TEXT,   -- JSON (list)
  per_core_freq    TEXT,   -- JSON (list)

  mem_percent      REAL,
  mem_total        INTEGER,
  mem_available    INTEGER,
  mem_used         INTEGER,
  mem_free         INTEGER,
  mem_cached       INTEGER,
  mem_buffers      INTEGER,
  mem_shared       INTEGER,

  swap_percent     REAL,
  swap_total       INTEGER,
  swap_used        INTEGER,
  swap_free        INTEGER,
  swap_sin         INTEGER,
  swap_sout        INTEGER,

  load1            REAL,
  load5            REAL,
  load15           REAL,

  hostname         TEXT,
  username         TEXT
);
CREATE INDEX IF NOT EXISTS idx_cpu_metrics_full_ts ON cpu_metrics_full(ts);
"""

def open_db(path: str):
    con = sqlite3.connect(path, isolation_level="DEFERRED", timeout=10.0)
    cur = con.cursor()
    # 軽い最適化
    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=NORMAL;")
    cur.execute("PRAGMA busy_timeout=5000;")
    cur.execute("PRAGMA temp_store=MEMORY;")
    cur.execute("PRAGMA mmap_size=134217728;")
    cur.executescript(DDL_MIN)
    cur.executescript(DDL_FULL)
    return con, cur

# 終了時クリーンアップ
_con = None
def _cleanup():
    global _con
    if _con is not None:
        try: _con.commit()
        except Exception: pass
        try: _con.close()
        except Exception: pass
atexit.register(_cleanup)

# CSVヘッダ（全項目）
FIELD_ORDER = [
    "ts","cpu_percent","temp_c",
    "cpu_user_pct","cpu_system_pct","cpu_idle_pct","cpu_iowait_pct",
    "cpu_ctx_switches","cpu_interrupts","cpu_soft_intr","cpu_syscalls",
    "cpu_logical","cpu_physical",
    "cpu_freq_mhz","cpu_freq_min_mhz","cpu_freq_max_mhz",
    "per_core_cpu","per_core_freq",
    "mem_percent","mem_total","mem_available","mem_used","mem_free","mem_cached","mem_buffers","mem_shared",
    "swap_percent","swap_total","swap_used","swap_free","swap_sin","swap_sout",
    "load1","load5","load15",
    "hostname","username",
]

# CSV表示：欠損は -1
def to_csv_cell(v):
    if isinstance(v, (list, dict)):
        return json.dumps(v, ensure_ascii=False)
    if v is None:
        return "-1"
    if isinstance(v, float) and (math.isnan(v) or math.isinf(v)):
        return "-1"
    return v

def to_record_dict(ts, cpu_total, temp,
                   cpu_user, cpu_sys, cpu_idle, cpu_iowait,
                   ctx_sw, intr, softintr, syscalls,
                   logical, physical,
                   fcur, fmn, fmx,
                   per_core_pct, per_core_freq,
                   mem_pct, mem_total, mem_avail, mem_used, mem_free, mem_cached, mem_buf, mem_share,
                   swap_pct, swap_total, swap_used, swap_free, swap_sin, swap_sout,
                   l1, l5, l15):
    return {
        "ts": ts,
        "cpu_percent": _f(cpu_total),
        "temp_c": _f(temp),

        "cpu_user_pct": _f(cpu_user),
        "cpu_system_pct": _f(cpu_sys),
        "cpu_idle_pct": _f(cpu_idle),
        "cpu_iowait_pct": _f(cpu_iowait),

        "cpu_ctx_switches": _i(ctx_sw),
        "cpu_interrupts": _i(intr),
        "cpu_soft_intr": _i(softintr),
        "cpu_syscalls": _i(syscalls),

        "cpu_logical": _i(logical),
        "cpu_physical": _i(physical),

        "cpu_freq_mhz": _f(fcur),
        "cpu_freq_min_mhz": _f(fmn),
        "cpu_freq_max_mhz": _f(fmx),

        "per_core_cpu": per_core_pct,
        "per_core_freq": per_core_freq,

        "mem_percent": _f(mem_pct),
        "mem_total": _i(mem_total),
        "mem_available": _i(mem_avail),
        "mem_used": _i(mem_used),
        "mem_free": _i(mem_free),
        "mem_cached": _i(mem_cached),
        "mem_buffers": _i(mem_buf),
        "mem_shared": _i(mem_share),

        "swap_percent": _f(swap_pct),
        "swap_total": _i(swap_total),
        "swap_used": _i(swap_used),
        "swap_free": _i(swap_free),
        "swap_sin": _i(swap_sin),
        "swap_sout": _i(swap_sout),

        "load1": _f(l1),
        "load5": _f(l5),
        "load15": _f(l15),

        "hostname": HOSTNAME,
        "username": USERNAME,
    }

def main():
    global _con
    con, cur = open_db(DB_PATH)
    _con = con

    writer = csv.writer(sys.stdout, lineterminator="\n")
    writer.writerow(FIELD_ORDER)

    inserted = 0
    next_tick = time.perf_counter()

    try:
        while not STOP:
            # 時刻同期
            now = time.perf_counter()
            if now < next_tick:
                time.sleep(next_tick - now)
            next_tick += INTERVAL

            # 計測（待たない）
            cpu_total = psutil.cpu_percent(interval=None)
            temp = get_cpu_temp_c()

            ct = psutil.cpu_times_percent(interval=0.0)
            cpu_user   = getattr(ct, "user",   None)
            cpu_sys    = getattr(ct, "system", None)
            cpu_idle   = getattr(ct, "idle",   None)
            cpu_iowait = getattr(ct, "iowait", None)

            cs = psutil.cpu_stats()
            ctx_sw   = getattr(cs, "ctx_switches",    None)
            intr     = getattr(cs, "interrupts",      None)
            softintr = getattr(cs, "soft_interrupts", None)
            syscalls = getattr(cs, "syscalls",        None)

            logical  = psutil.cpu_count(logical=True)
            physical = psutil.cpu_count(logical=False)

            f   = psutil.cpu_freq()
            fmn = f.min if f else None
            fmx = f.max if f else None
            fcur= f.current if f else None

            per_core_pct  = psutil.cpu_percent(percpu=True, interval=None)
            try:
                _pcf = psutil.cpu_freq(percpu=True)
                per_core_freq = [_f(x.current) for x in _pcf] if _pcf else None
            except Exception:
                per_core_freq = None

            vm = psutil.virtual_memory()
            mem_pct   = vm.percent
            mem_total = vm.total
            mem_avail = vm.available
            mem_used  = vm.used
            mem_free  = vm.free
            mem_cached= getattr(vm, "cached",  None)
            mem_buf   = getattr(vm, "buffers", None)
            mem_share = getattr(vm, "shared",  None)

            sw = psutil.swap_memory()
            swap_pct  = sw.percent
            swap_total= sw.total
            swap_used = sw.used
            swap_free = sw.free
            swap_sin  = getattr(sw, "sin",  None)
            swap_sout = getattr(sw, "sout", None)

            try:
                l1, l5, l15 = os.getloadavg()
            except Exception:
                l1 = l5 = l15 = None

            ts = now_iso()

            # --- DB: 4列ミニ（列名明示） ---
            try:
                cur.execute(
                    "INSERT INTO cpu_metrics_min (ts, cpu_percent, cpu_ctx_switches, temp_c) VALUES (?,?,?,?)",
                    (ts, _f(cpu_total), _i(ctx_sw), _f(temp))
                )
                inserted += 1
                if inserted % BATCH_COMMIT == 0:
                    con.commit()
            except Exception as e:
                print(f"[WARN] insert(min) failed at {ts}: {e}", file=sys.stderr)

            # --- DB: 全項目（列名明示・37個の?） ---
            try:
                cur.execute("""
                    INSERT INTO cpu_metrics_full (
                      ts, cpu_percent, temp_c,
                      cpu_user_pct, cpu_system_pct, cpu_idle_pct, cpu_iowait_pct,
                      cpu_ctx_switches, cpu_interrupts, cpu_soft_intr, cpu_syscalls,
                      cpu_logical, cpu_physical,
                      cpu_freq_mhz, cpu_freq_min_mhz, cpu_freq_max_mhz,
                      per_core_cpu, per_core_freq,
                      mem_percent, mem_total, mem_available, mem_used, mem_free, mem_cached, mem_buffers, mem_shared,
                      swap_percent, swap_total, swap_used, swap_free, swap_sin, swap_sout,
                      load1, load5, load15,
                      hostname, username
                    ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                """, (
                    ts, _f(cpu_total), _f(temp),
                    _f(cpu_user), _f(cpu_sys), _f(cpu_idle), _f(cpu_iowait),
                    _i(ctx_sw), _i(intr), _i(softintr), _i(syscalls),
                    _i(logical), _i(physical),
                    _f(fcur), _f(fmn), _f(fmx),
                    _json(per_core_pct), _json(per_core_freq),
                    _f(mem_pct), _i(mem_total), _i(mem_avail), _i(mem_used), _i(mem_free), _i(mem_cached), _i(mem_buf), _i(mem_share),
                    _f(swap_pct), _i(swap_total), _i(swap_used), _i(swap_free), _i(swap_sin), _i(swap_sout),
                    _f(l1), _f(l5), _f(l15),
                    HOSTNAME, USERNAME
                ))
                if inserted % BATCH_COMMIT == 0:
                    con.commit()
            except Exception as e:
                print(f"[WARN] insert(full) failed at {ts}: {e}", file=sys.stderr)

            # --- CSV: 全項目1行出力（None/NaN/Inf → -1） ---
            rec = to_record_dict(
                ts, cpu_total, temp,
                cpu_user, cpu_sys, cpu_idle, cpu_iowait,
                ctx_sw, intr, softintr, syscalls,
                logical, physical,
                fcur, fmn, fmx,
                per_core_pct, per_core_freq,
                mem_pct, mem_total, mem_avail, mem_used, mem_free, mem_cached, mem_buf, mem_share,
                swap_pct, swap_total, swap_used, swap_free, swap_sin, swap_sout,
                l1, l5, l15
            )
            row = [to_csv_cell(rec.get(k)) for k in FIELD_ORDER]
            writer.writerow(row)
            sys.stdout.flush()

    except KeyboardInterrupt:
        pass
    finally:
        try: con.commit()
        except Exception: pass
        try: con.close()
        except Exception: pass

if __name__ == "__main__":
    main()


ts,cpu_percent,temp_c,cpu_user_pct,cpu_system_pct,cpu_idle_pct,cpu_iowait_pct,cpu_ctx_switches,cpu_interrupts,cpu_soft_intr,cpu_syscalls,cpu_logical,cpu_physical,cpu_freq_mhz,cpu_freq_min_mhz,cpu_freq_max_mhz,per_core_cpu,per_core_freq,mem_percent,mem_total,mem_available,mem_used,mem_free,mem_cached,mem_buffers,mem_shared,swap_percent,swap_total,swap_used,swap_free,swap_sin,swap_sout,load1,load5,load15,hostname,username
2025-10-28T03:53:23.609539Z,60.7,-1,52.2,7.5,39.3,-1,350790488,325810371,0,1628409640,4,2,1209.0,0.0,1512.0,"[60.8, 59.0, 61.0, 61.9]",[1209.0],79.3,8473911296,1753567232,6720344064,1753567232,-1,-1,-1,16.5,5637144576,927313920,4709830656,0,0,-1,-1,-1,DESKTOP-8FOAVTD,flare
2025-10-28T03:53:24.611745Z,43.6,-1,27.0,15.4,56.4,-1,350800704,325818390,0,1628456463,4,2,1209.0,0.0,1512.0,"[54.5, 40.0, 42.2, 37.5]",[1209.0],79.3,8473911296,1752502272,6721409024,1752502272,-1,-1,-1,16.4,5637144576,927309824,4709834752,0,0,-1,-1,-1,DESKTOP-8FOAVTD,flare
2025-10-28T03:53:25.624657Z